In [1]:
import os
import numpy as np
import glob
import shutil
import xarray as xr
from astropy.time import Time
import museval
from museval.utils import get_response, find_response, aia_synthesis, wavelength_in_cube
from museval.io import create_session, is_complete
from muse.synthesis.synthesis import transform_resp_units, vdem_synthesis
from matplotlib import colors
import matplotlib.pyplot as plt
from muse.instr.utils import convert_resp2muse_ciresp
import astropy.units as u
from matplotlib.ticker import PercentFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.wcs import WCS
from astropy.time import Time
import sunpy.map
import sunpy.visualization.colormaps as cm
from sunpy.time import parse_time
from dateutil.tz import gettz
import astropy.io.ascii as ascii
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import eispac
from museval.eis_calibration.eis_calib_2023 import calib_2023 as eis_calib_2023
from matplotlib import gridspec
import warnings
warnings.filterwarnings("ignore", category=UserWarning, append=True)
warnings.filterwarnings(
    "ignore",
    message="Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata.*",
    category=RuntimeWarning
) #To ignore this  annoying Zarr warning

In [3]:
os.environ['text_files_path'] = '/Users/souvikb/various_analysis/GaussSep/bose_codes/'
os.environ['eis_data_path'] = '/Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/'
txt_files = glob.glob(os.path.join(os.environ['text_files_path'], "*.txt"))
txt_files = sorted(txt_files)

In [4]:
obs_dates = ascii.read(txt_files[0])
obs_dates.add_index("date_begin_EIS")
for date in tqdm(obs_dates['date_begin_EIS']):
    date_begin_EIS = obs_dates.loc[date]["date_begin_EIS"]
    print(f'Processing EIS data for date: {date_begin_EIS}')
    cutouts_data_path = os.path.join(os.environ['eis_data_path'], "SDO_EIS_cutouts_"+date_begin_EIS)
    downloaded_data_h5 = glob.glob(cutouts_data_path+'/*.data.h5')
    downloaded_head_h5 = glob.glob(cutouts_data_path+'/*.head.h5')
    #check if the variables are empty or not before proceeding with the next steps
    if not downloaded_data_h5 or not downloaded_head_h5:
        print(f"No data or header files found in {cutouts_data_path}")
        continue
    else:
        data = eispac.read_cube(downloaded_data_h5[0])
        headers = eispac.read_cube(downloaded_head_h5[0])
        eis_lines = [195.120,197.860, 284.160] # Fe XII, Fe IX and Fe XV
        target_template_name = ['fe_12_195_119.2c.template.h5', 'fe_09_197_862.1c.template.h5','fe_15_284_160.2c.template.h5']
        eis_cmap = ['sdoaia193','Blues_r','sohoeit284']
        eis_list_data = [] # To check how many if not all lines listed above are available in the actual EIS data
        data_intensity_maps = [] # To store the intensity maps for all the available lines in the actual EIS data.
        for target_template, eis_spect in zip(target_template_name,eis_lines):
            eis_list_data.append(wavelength_in_cube(downloaded_data_h5[0],str(eis_spect)))
            if wavelength_in_cube(downloaded_data_h5[0], str(eis_spect)):
                data_cube = eispac.read_cube(downloaded_data_h5[0], eis_spect, apply_radcal=True)
                template_list = eispac.match_templates(data_cube)
                index = next(i for i, path in enumerate(template_list) if path.name == target_template)
                shutil.copy(template_list[index], cutouts_data_path)
                if __name__ == '__main__':
                    template = glob.glob(os.path.join(cutouts_data_path,target_template))#'fe_12*.template.h5'))
                    template_filepath = template[0]
                    tmplt = eispac.read_template(template_filepath)
                    # Read spectral window into an EISCube
                    data_cube = eispac.read_cube(downloaded_data_h5[0], tmplt.central_wave,apply_radcal=True)
                    fit_res = eispac.fit_spectra(data_cube, tmplt, ncpu='max')
                    inten_map = fit_res.get_map(component=0, measurement='intensity')
                    inten_map = eis_calib_2023(inten_map)
                    data_intensity_maps.append(inten_map.data) # Collecting the intensity arrays for plotting the histograms later
                    vel_map = fit_res.get_map(component=0, measurement='velocity')
                    # wd_map = fit_res.get_map(component=0, measurement='width')
                        # Get one map to measure aspect ratio
                    sample_map = inten_map  # or any map with the correct shape
                    ny, nx = sample_map.data.shape
                    aspect = ny / nx  # height / width
                    panel_width = 4  # inches per panel (adjust as needed)
                    panel_height = panel_width * aspect
                    fig_width = panel_width * 2.4  # extra space for colorbars
                    # fig_width = panel_width * 2  # two panels side by side
                    fig_height = panel_height  # two panels stacked vertically
                    # fig_height = panel_width * aspect/8.  # maintain aspect
                    fig = plt.figure(figsize=(fig_width, fig_height)) 
                    gs = gridspec.GridSpec(nrows=1, ncols=2, width_ratios=[1, 1], wspace=0.25)
                    ax1 = fig.add_subplot(gs[0], projection=inten_map)
                    if target_template == 'fe_12_195_119.2c.template.h5':
                        # ax1 = fig.add_subplot(121,projection=inten_map)
                        im=inten_map.plot(axes=ax1, cmap = eis_cmap[0])
                        cbar =fig.colorbar(im,extend='both',fraction=0.05,pad=0.05)
                        cbar.ax.set_ylabel(r'[erg s$^{-1}$ cm$^{-2}$ s$^{-1}$]')
                        ax1.set_title(f'Fe XII {eis_spect}',size=10)
                        ax1.grid(False)
                    elif target_template == 'fe_09_197_862.1c.template.h5':
                        # ax1 = fig.add_subplot(121,projection=inten_map)
                        im=inten_map.plot(axes=ax1, cmap = eis_cmap[1])
                        cbar=fig.colorbar(im,extend='both',fraction=0.05,pad=0.05)
                        cbar.ax.set_ylabel(r'[erg s$^{-1}$ cm$^{-2}$ s$^{-1}$]')
                        ax1.set_title(f'Fe IX {eis_spect}',size=10)
                        ax1.grid(False)
                    else:
                        # ax1 = fig.add_subplot(121,projection=inten_map)
                        im=inten_map.plot(axes=ax1, cmap = eis_cmap[2])
                        cbar=fig.colorbar(im,fraction=0.05,pad=0.05)
                        cbar.ax.set_ylabel(r'[erg s$^{-1}$ cm$^{-2}$ s$^{-1}$]')
                        ax1.set_title(f'Fe XV {eis_spect}',size=10)
                        ax1.grid(False)
                    
                    vel_map.plot_settings["norm"]=None
                    ax2 = fig.add_subplot(gs[1],projection=vel_map)
                    im=vel_map.plot(axes=ax2,vmax=10,vmin=-10,cmap='coolwarm')
                    cbar=fig.colorbar(im,extend ='both', fraction=0.046,pad=0.05)
                    cbar.ax.set_ylabel(r'[kms$^{-1}$]')
                    ax2.set_title(r'Core shift',size=10)
                    ax2.grid(False)
                    plt.tight_layout()
                    # plt.show()
                    os.chdir(cutouts_data_path)
                    output_dir = './pipeline_figs'
                    os.makedirs(output_dir, exist_ok=True)
                    print('Saving the EIS maps')
                    fig.savefig(os.path.join(output_dir,f"{date}_EIS_{eis_spect}.png"),dpi=300,bbox_inches='tight')
                    plt.close(fig)
            else:
                print(f"Skipping wavelength {eis_spect} Å: Not found in {downloaded_data_h5[0]}")
    # Plotting the EIS intensity histograms
    n_panels_eis_hist = sum(eis_list_data)
    fig, ax = plt.subplots(nrows=1, ncols=n_panels_eis_hist, figsize=(12, 4.))
    ax = np.atleast_1d(ax)
    eis_hist_bins = 60 # hard coded for now
    for i, (eis_spect, intensity_map) in enumerate(zip(eis_lines, data_intensity_maps)):
        if eis_list_data[i]:
            # Get the intensity data for the current line
            intensity_data = intensity_map.ravel()
            min_int = max(intensity_data.min(), 1e-1)  # avoid log(0)
            max_int = intensity_data.max()
            # Create histogram bins
            bins = np.logspace(np.log10(min_int), np.log10(max_int), eis_hist_bins)
            # Plot histogram
            ax[i].hist(intensity_data, bins=bins, histtype='step', cumulative=False,
                        weights=np.ones(len(intensity_data)) / len(intensity_data), color='blue')
            ax[i].set_xlabel(r'Intensity [ergs cm$^{-2}$ s$^{-1}$ sr$^{-1}$]')
            ax[i].set_ylabel('Fraction')
            ax[i].axvline(x=np.mean(intensity_data), color='red', lw=1, ls='-',label='Mean')
            ax[i].legend(loc='upper left')
            ax[i].set_title(f'EIS {eis_spect} {date_begin_EIS}',size=10)

            color='tab:green'
            axs_twin=ax[i].twinx()
            axs_twin.hist(intensity_data, bins=bins, histtype='step', cumulative=True,
                        weights=np.ones(len(intensity_data)) / len(intensity_data), color='green', ls='-')
            axs_twin.axhline(y=0.25, linestyle='dashed',color='green')
            axs_twin.axhline(y=0.75, linestyle='dashed',color='green')
            axs_twin.set_ylabel('ECDF',color='green')
            axs_twin.tick_params(axis='y', labelcolor=color)
        else:
            print(f"Skipping wavelength {eis_spect} Å: Not found in {downloaded_data_h5[0]}")
    
    # Adjust layout and show the plot
    plt.tight_layout()
    os.chdir(cutouts_data_path)
    output_dir = './pipeline_figs'
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(os.path.join(output_dir,f"{date}_EIS_intensity_histograms.png"), dpi=300, bbox_inches='tight')
    plt.close(fig)

    print('Now plotting the HMI LOS flux density cutouts for the same date and region...')
    ## Plotting the HMI LOS flux density cutouts
    # First grab the cutout region from the full disk HMI data
    bottom_left = SkyCoord(data.meta['extent_arcsec'][0]*u.arcsec, data.meta['extent_arcsec'][2]*u.arcsec, obstime=data.meta['mod_index']['date_obs'], observer="earth", frame="helioprojective")
    top_right = SkyCoord(data.meta['extent_arcsec'][1]*u.arcsec, data.meta['extent_arcsec'][3]*u.arcsec, obstime=data.meta['mod_index']['date_obs'], observer="earth", frame="helioprojective")

    ## Load the HMI data as sunpy map
    hmi_map = sunpy.map.Map(cutouts_data_path+'/*magnetogram.fits') ## This will  be rotated by 90 degrees as usual
    ## Load an AIA map to get the WCS
    aia_map_fdisk = sunpy.map.Map(cutouts_data_path+'/*.193.image_lev1.fits')
    out_hmi = hmi_map.reproject_to(aia_map_fdisk.wcs) ## actual reprojection from HMI to AIA based on WCS headers
    cutout_hmi_aligned = out_hmi.submap(bottom_left,top_right=top_right) ## this is the cutout 
    ## Now dynamically calculate figure size based on the number of subplots and aspect ratio
    ny, nx = cutout_hmi_aligned.data.shape
    aspect = ny / nx
    panel_width = 4
    fig_width = panel_width * 2
    fig_height = aspect*fig_width*0.9
    cutout_hmi_aligned.plot_settings["norm"]=None

    fig= plt.figure(figsize=(fig_width,fig_height))
    ax = fig.add_subplot(2, 2, 1, projection=cutout_hmi_aligned.wcs)
    im = cutout_hmi_aligned.plot(axes=ax, vmax=0.3*np.max(cutout_hmi_aligned.data), vmin=-0.3*np.max(cutout_hmi_aligned.data))
    ax.grid(False)
    lon = ax.coords[0]
    lat = ax.coords[1]
    lon.set_ticks_position('b')      # ticks on bottom
    lon.set_ticklabel_position('b')  # bottom
    lat.set_ticks_position('l')      # ticks on left
    lat.set_ticklabel_position('l')  # left
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05, axes_class=plt.Axes)
    cbar = fig.colorbar(im, cax=cax,extend='both')
    cbar.ax.tick_params(direction='out')
    cbar.ax.yaxis.set_ticks_position('right')
    cbar.ax.yaxis.set_label_position('right')
    cbar.ax.yaxis.tick_right()
    cbar.set_label(r'B$_{\mathrm{LOS}}$[G]')

    ax = fig.add_subplot(2, 2, 2, projection=cutout_hmi_aligned.wcs)
    abs_data = np.abs(cutout_hmi_aligned.data) 
    abs_map = sunpy.map.Map(abs_data, cutout_hmi_aligned.meta)
    im = abs_map.plot(axes=ax, clip_interval=(1, 99.99)*u.percent,cmap='YlGnBu')
    ax.grid(False)
    lon = ax.coords[0]
    lat = ax.coords[1]
    lon.set_ticks_position('b')      # ticks on bottom
    lon.set_ticklabel_position('b')  # bottom
    lat.set_ticks_position('l')      # ticks on left
    lat.set_ticklabel_position(' ')  # left
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05, axes_class=plt.Axes)
    cbar = fig.colorbar(im, cax=cax, extend ='both')
    cbar.ax.tick_params(direction='out')
    cbar.ax.yaxis.set_ticks_position('right')
    cbar.ax.yaxis.set_label_position('right')
    cbar.ax.yaxis.tick_right()
    cbar.set_label(r'|B$_{\mathrm{LOS}}$| [G]')
    #---- Histograms ------#
    bins = np.linspace(cutout_hmi_aligned.data.min(),cutout_hmi_aligned.data.max(),100)
    data_hmi = cutout_hmi_aligned.data
    ax = fig.add_subplot(2, 2, 3)
    ax.hist(data_hmi.ravel(),bins=bins, histtype='step',weights=np.ones(len(data_hmi.ravel()))/len(data_hmi.ravel()),log=True,color='blue')
    ax.set_ylabel('Fraction')
    ax.set_xlabel(r'B$_{\mathrm{LOS}}$[G]')
    ax.axvline(x=0,ls='-',color='black')
    
    bins = np.linspace(abs_data.min(),abs_data.max(),100)
    # data_hmi = cutout_hmi_aligned.data
    ax = fig.add_subplot(2, 2, 4)
    ax.hist(abs_data.ravel(),bins=bins, histtype='step',weights=np.ones(len(abs_data.ravel()))/len(abs_data.ravel()),log=True,color='blue')
    # ax.set_ylabel('Fraction')
    ax.set_xlabel(r'|B$_{\mathrm{LOS}}$| [G]')
    plt.subplots_adjust(left=0.1,right=0.9,top=0.9,bottom=0.1,wspace=0.3,hspace=0.2)
    ax.axvline(x=np.mean(abs_data),ls='--',color='black',label=f'$\mu$: {np.mean(abs_data):.2f} [G]')
    ax.legend(loc='best')
    # plt.show()
    # save_hmi = input("Do you want to save the HMI maps? (True/False): ")
    # Ensure the directory exists
    os.chdir(cutouts_data_path)
    output_dir = './pipeline_figs'
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(os.path.join(output_dir,f"{date}_HMI.png"),dpi=300,bbox_inches='tight') 
    plt.close(fig)

    ## Plot a fukll disk AIA map and draw the cutout region
    exp_time_fdisk = aia_map_fdisk.exposure_time
    data_fdisk = aia_map_fdisk/exp_time_fdisk
    fig = plt.figure()
    ax = fig.add_subplot(projection=data_fdisk)
    im = data_fdisk.plot(axes=ax, vmin=0)
    ax.grid(False)
    divider= make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05, axes_class=plt.Axes)
    cbar = fig.colorbar(im, cax=cax)
    cbar.ax.tick_params(direction='out')
    cbar.ax.yaxis.set_ticks_position('right')
    cbar.ax.yaxis.set_label_position('right')
    cbar.ax.yaxis.tick_right()
    cbar.set_label('DN/s')
    cutout_hmi_aligned.draw_extent(axes=ax)
    # plt.show()
    os.chdir(cutouts_data_path)
    output_dir = './pipeline_figs'
    os.makedirs(output_dir, exist_ok=True)
    fig.savefig(os.path.join(output_dir,f"{date}_AIA_fdisk.png"),dpi=300,bbox_inches='tight')
    plt.close(fig)



  0%|          | 0/1 [00:00<?, ?it/s]

Processing EIS data for date: 2017-07-29T23:49:52
Data file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.data.h5
Header file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.head.h5
Found window 0
Data file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.data.h5
Header file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.head.h5
Found window 0
Data file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.data.h5
Header file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.head.h5
Found a wavelength 195.12 

/Users/souvikb/miniforge3/envs/muse_bose/lib/python3.11/site-packages/eispac/core/fit_spectra.py:428: RuntimeWarning: invalid value encountered in divide
  rel_err = obs_errs/obs_cent
/Users/souvikb/miniforge3/envs/muse_bose/lib/python3.11/site-packages/eispac/core/fit_spectra.py:428: RuntimeWarning: invalid value encountered in divide
  rel_err = obs_errs/obs_cent




Finished computing fits!
   runtime : 0:00:19.036809
   44542 spectra fit without issues
   2 spectra have < 7 good data points
   0 spectra have bad or invalid parameters
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
Saving the EIS maps
Data file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.data.h5
Header file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.head.h5
Found a wavelength 197.86 [Angstroms] in window 9
Data file,
   /Use

/Users/souvikb/miniforge3/envs/muse_bose/lib/python3.11/site-packages/eispac/core/fit_spectra.py:428: RuntimeWarning: invalid value encountered in divide
  rel_err = obs_errs/obs_cent




Finished computing fits!
   runtime : 0:00:16.941809
   44454 spectra fit without issues
   90 spectra have < 7 good data points
   0 spectra have bad or invalid parameters
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
Saving the EIS maps
Data file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.data.h5
Header file,
   /Users/souvikb/various_analysis/GaussSep/bose_codes/EIS_SDO_IRIS_data/SDO_EIS_cutouts_2017-07-29T23:49:52/eis_20170729_235915.head.h5
Found a wavelength 284.16 [Angstroms] in window 24
Data file,
   /U

/Users/souvikb/miniforge3/envs/muse_bose/lib/python3.11/site-packages/eispac/core/fit_spectra.py:428: RuntimeWarning: invalid value encountered in divide
  rel_err = obs_errs/obs_cent
/Users/souvikb/miniforge3/envs/muse_bose/lib/python3.11/site-packages/eispac/core/fit_spectra.py:428: RuntimeWarning: invalid value encountered in divide
  rel_err = obs_errs/obs_cent




Finished computing fits!
   runtime : 0:00:43.721895
   44032 spectra fit without issues
   512 spectra have < 7 good data points
   0 spectra have bad or invalid parameters
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
Saving the EIS maps
Now plotting the HMI LOS flux density cutouts for the same date and region...


100%|██████████| 1/1 [01:38<00:00, 98.51s/it]
